# Data Tidying Notebook

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re

import warnings
warnings.filterwarnings('ignore')

In [2]:
# Load dataset
df_main = pd.read_csv('developer_dataset.csv')
df_main.head()

,RespondentID,Year,Country,Employment,UndergradMajor,DevType,LanguageWorkedWith,LanguageDesireNextYear,DatabaseWorkedWith,DatabaseDesireNextYear,...,PlatformDesireNextYear,Hobbyist,OrgSize,YearsCodePro,JobSeek,ConvertedComp,WorkWeekHrs,NEWJobHunt,NEWJobHuntResearch,NEWLearn
0,1,2018,United States,Employed full-time,"Computer science, computer engineering, or sof...",Engineering manager;Full-stack developer,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,141000.0,NaN,NaN,NaN,NaN
1,1,2019,United States,Employed full-time,"Computer science, computer engineering, or sof...","Developer, full-stack",C;C++;C#;Python;SQL,C;C#;JavaScript;SQL,MySQL;SQLite,MySQL;SQLite,...,Linux;Windows,No,100 to 499 employees,1.0,I am not interested in new job opportunities,61000.0,80.0,NaN,NaN,NaN
2,1,2020,United States,Employed full-time,"Computer science, computer engineering, or sof...",NaN,HTML/CSS;Ruby;SQL,Java;Ruby;Scala,MySQL;PostgreSQL;Redis;SQLite,MySQL;PostgreSQL,...,Docker;Google Cloud Platform;Heroku;Linux;Windows,Yes,NaN,8.0,NaN,NaN,NaN,NaN,NaN,Once a year
3,2,2018,United States,Employed full-time,"Computer science, computer engineering, or sof...",Full-stack developer,C#;JavaScript;SQL;TypeScript;HTML;CSS;Bash/Shell,C#;JavaScript;SQL;TypeScript;HTML;CSS;Bash/Shell,"SQL Server;Microsoft Azure (Tables, CosmosDB, ...","SQL Server;Microsoft Azure (Tables, CosmosDB, ...",...,Azure,NaN,NaN,4.0,NaN,48000.0,NaN,NaN,NaN,NaN
4,2,2019,United States,Employed full-time,"Computer science, computer engineering, or sof...",Data or business analyst;Database administrato...,Bash/Shell/PowerShell;HTML/CSS;JavaScript;PHP;...,Bash/Shell/PowerShell;HTML/CSS;JavaScript;Rust...,Couchbase;DynamoDB;Firebase;MySQL,Firebase;MySQL;Redis,...,Android;AWS;Docker;IBM Cloud or Watson;Linux;S...,Yes,10 to 19 employees,8.0,I am not interested in new job opportunities,90000.0,40.0,NaN,NaN,NaN


In [3]:
# Missing values rate
1 - (df_main.count().sort_values()) / len(df_main.RespondentID)

NEWJobHuntResearch        0.832001
NEWJobHunt                0.828009
NEWLearn                  0.782158
WorkWeekHrs               0.540604
OrgSize                   0.507198
JobSeek                   0.455476
Hobbyist                  0.385373
DatabaseDesireNextYear    0.332482
PlatformDesireNextYear    0.232292
DatabaseWorkedWith        0.227949
ConvertedComp             0.178727
PlatformWorkedWith        0.176245
YearsCodePro              0.147614
LanguageDesireNextYear    0.136365
UndergradMajor            0.114703
DevType                   0.096899
LanguageWorkedWith        0.082646
Employment                0.016042
Country                   0.000000
Year                      0.000000
RespondentID              0.000000
dtype: float64

In [4]:
# Drop columns where 60% values are NA
df_main = df_main.dropna(thresh=len(df_main.RespondentID)*0.4, axis=1)

In [5]:
# View NA values after dropping NA values
1 - (df_main.count().sort_values()) / len(df_main.RespondentID)

WorkWeekHrs               0.540604
OrgSize                   0.507198
JobSeek                   0.455476
Hobbyist                  0.385373
DatabaseDesireNextYear    0.332482
PlatformDesireNextYear    0.232292
DatabaseWorkedWith        0.227949
ConvertedComp             0.178727
PlatformWorkedWith        0.176245
YearsCodePro              0.147614
LanguageDesireNextYear    0.136365
UndergradMajor            0.114703
DevType                   0.096899
LanguageWorkedWith        0.082646
Employment                0.016042
Country                   0.000000
Year                      0.000000
RespondentID              0.000000
dtype: float64

In [6]:
# Iterate through columns to ID unique values
# for col in df_main.columns:
#     print(f'\n {col} :\n {df_main.loc[:,col].value_counts(dropna=False, normalize=True)}')

In [ ]:
# Cleaning by Highest % NaN
# Safe assumption that hours per week above 200 were mistyped, and safe to divide by 10. Round all values to nearest 5th.
for index, row in df_main.iterrows():
    if row['WorkWeekHrs'] > 200:
        df_main.at[index, 'WorkWeekHrs'] = df_main.at[index, 'WorkWeekHrs']/ 10
    df_main.at[index, 'WorkWeekHrs'] = 5 * round(df_main.at[index, 'WorkWeekHrs']/5, 0)
# df_main.WorkWeekHrs.value_counts(dropna=False, normalize=True)
sns.displot(df_main.WorkWeekHrs)

In [ ]:
# Checking OrgSize
# df_main.OrgSize.value_counts(dropna=False, normalize=True)
df_main.drop('OrgSize', inplace=True, axis=1)

In [ ]:
# Checking JobSeek
df_main.JobSeek.value_counts(dropna=False, normalize=True)
sns.displot(df_main.JobSeek)

In [ ]:
# Hobbyist - not interested 
df_main.drop('Hobbyist', inplace=True, axis=1)

In [ ]:
# Allow for one response only in DatabaseDesireNextYear
# df_main['DatabaseDesireNextYear'] = df_main['DatabaseDesireNextYear'].apply(lambda x: x if pd.isna(x) else re.sub(r';.*$', '', x))

In [ ]:
# Allow for one response only in PlatformDesireNextYear
# df_main['PlatformDesireNextYear'] = df_main['PlatformDesireNextYear'].apply(lambda x: x if pd.isna(x) else re.sub(r';.*$', '', x))

In [ ]:
# Checking DatabaseWorkedWith, allowing for one resposne
# df_main['DatabaseWorkedWith'] = df_main['DatabaseWorkedWith'].apply(lambda x: x if pd.isna(x) else re.sub(r';.*$', '', x))

In [ ]:
# Convert ConvertedComp to the nearest 100000
df_main.ConvertedComp.value_counts(dropna=False, normalize=True)
for index, row in df_main.iterrows():
    df_main.at[index, 'ConvertedComp'] = 10000 * round(df_main.at[index, 'ConvertedComp']/10000, 0)

In [ ]:
# Checking PlatformWorkedWith, allow for one response
# df_main['PlatformWorkedWith'] = df_main['PlatformWorkedWith'].apply(lambda x: x if pd.isna(x) else re.sub(r';.*$', '', x))

In [ ]:
# YearsCodePro
df_main.YearsCodePro.value_counts(dropna=False, normalize=True)
sns.displot(df_main.YearsCodePro)
plt.show()
plt.clf()

In [ ]:
df_main.YearsCodePro.value_counts(dropna=False, normalize=True)

In [ ]:
# Checking LanguageDesireNextYear, allow for one response
# df_main['LanguageDesireNextYear'] = df_main['LanguageDesireNextYear'].apply(lambda x: x if pd.isna(x) else re.sub(r';.*$', '', x))
df_main.LanguageDesireNextYear.value_counts(dropna=False, normalize=True)

In [ ]:
# Checkign UndergradMajor, renaming for clarity
df_main['UndergradMajor'] = df_main['UndergradMajor'].replace('Computer science, computer engineering, or software engineering', 'CompSci')
df_main['UndergradMajor'] = df_main['UndergradMajor'].replace('Information systems, information technology, or system administration', 'IT')
df_main['UndergradMajor'] = df_main['UndergradMajor'].replace('Mathematics or statistics', 'Mathematics')
df_main['UndergradMajor'] = df_main['UndergradMajor'].replace('A natural science (ex. biology, chemistry, physics)', 'Natural Sciences')
df_main['UndergradMajor'] = df_main['UndergradMajor'].replace('A natural science (such as biology, chemistry, physics, etc.)', 'Natural Sciences')
df_main['UndergradMajor'] = df_main['UndergradMajor'].replace('A health science (ex. nursing, pharmacy, radiology)', 'Health Sciences')
df_main['UndergradMajor'] = df_main['UndergradMajor'].replace('A health science (such as nursing, pharmacy, radiology, etc.)', 'Health Sciences')
df_main['UndergradMajor'] = df_main['UndergradMajor'].replace('A social science (such as anthropology, psychology, political science, etc.)', 'Social Sciences')
df_main['UndergradMajor'] = df_main['UndergradMajor'].replace('Fine arts or performing arts (such as graphic design, music, studio art, etc.)', 'Arts')
df_main['UndergradMajor'] = df_main['UndergradMajor'].replace('Web development or web design', 'UI/ UX')
df_main['UndergradMajor'] = df_main['UndergradMajor'].replace('A humanities discipline (ex. literature, history, philosophy)', 'Humanities')
df_main['UndergradMajor'] = df_main['UndergradMajor'].replace('A social science (ex. anthropology, psychology, political science) ', 'Social Sciences')
df_main['UndergradMajor'] = df_main['UndergradMajor'].replace('Fine arts or performing arts (ex. graphic design, music, studio art)', 'Arts')
df_main['UndergradMajor'] = df_main['UndergradMajor'].replace('A business discipline (ex. accounting, finance, marketing)', 'Business')
df_main['UndergradMajor'] = df_main['UndergradMajor'].replace('A business discipline (such as accounting, finance, marketing, etc.)', 'Business')
df_main['UndergradMajor'] = df_main['UndergradMajor'].replace('A social science (ex. anthropology, psychology, political science)', 'Social Sciences')
df_main['UndergradMajor'] = df_main['UndergradMajor'].replace('Another engineering discipline (such as civil, electrical, mechanical, etc.)', 'Engineering')
df_main['UndergradMajor'] = df_main['UndergradMajor'].replace('Another engineering discipline (ex. civil, electrical, mechanical)', 'Engineering')
df_main['UndergradMajor'] = df_main['UndergradMajor'].replace('A humanities discipline (such as literature, history, philosophy, etc.)', 'Humanities')
df_main['UndergradMajor'] = df_main['UndergradMajor'].replace('I never declared a major', 'Undeclared')
df_main.UndergradMajor.value_counts(dropna=False, normalize=True)

In [ ]:
# Checking DevType, keep first, aggregate
df_main['DevType'] = df_main['DevType'].apply(lambda x: x if pd.isna(x) else re.sub(r';.*$', '', x))
df_main['DevType'] = df_main['DevType'].replace('Back-end developer', 'Developer, back-end')
df_main['DevType'] = df_main['DevType'].replace('Full-stack developer', 'Developer, full-stack')
df_main['DevType'] = df_main['DevType'].replace('Scientist', 'Academic researcher')
df_main['DevType'] = df_main['DevType'].replace('Front-end developer', 'Developer, front-end')
df_main['DevType'] = df_main['DevType'].replace('Front-end developer', 'Developer, front-end')
df_main['DevType'] = df_main['DevType'].replace('Data scientist or machine learning specialist', 'Data Scientist')
df_main['DevType'] = df_main['DevType'].replace('Desktop or enterprise applications developer', 'Developer, desktop or enterprise applications')
df_main['DevType'] = df_main['DevType'].replace('Mobile developer', 'Developer, mobile')
df_main['DevType'] = df_main['DevType'].replace('Embedded applications or devices developer', 'Developer, embedded applications or devices')
df_main['DevType'] = df_main['DevType'].replace('Educator', 'Academic researcher')
df_main['DevType'] = df_main['DevType'].replace('QA or test developer', 'Developer, QA or test')
df_main['DevType'] = df_main['DevType'].replace('Product manager', 'Academic researcher')
df_main['DevType'] = df_main['DevType'].replace('Educator or academic researcher', 'Marketing or sales professional')
df_main['DevType'] = df_main['DevType'].replace('Educator or academic researcher', 'Academic researcher')
df_main.DevType.value_counts(dropna=False, normalize=True)

In [ ]:
# Checking LanguageWorkedWith, again dropping except first
df_main['LanguageWorkedWith'] = df_main['LanguageWorkedWith'].apply(lambda x: x if pd.isna(x) else re.sub(r';.*$', '', x))
df_main.LanguageWorkedWith.value_counts(dropna=False, normalize=True)

In [ ]:
# Before dropping additional values
df_main.info()

In [ ]:
# Drop rows with majority NaN Information
df_main = df_main.dropna(thresh=len(df_main.columns)*0.6, axis=0)
df_main.info()

In [ ]:
# Missing Data Rate
1 - (df_main.count().sort_values()) / len(df_main.RespondentID)

In [ ]:
df_main.to_csv('developer_dataset_clean.csv', encoding='utf8')

In [ ]:
df_main.head()